<a href="https://colab.research.google.com/github/UzunDemir/9_matrix_knn/blob/main/%D0%A1%D0%B0%D0%BC%D0%BE%D0%BF%D1%80%D0%BE%D0%B2%D0%B5%D1%80%D0%BA%D0%B0_matrix_knn_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import numpy as np
import pandas as pd

In [ ]:
books = pd.read_csv('books.csv')

In [ ]:
ratings = pd.read_csv('ratings.csv')
ratings

,user_id,book_id,rating
0,1,258,5
1,2,4081,4
2,2,260,5
3,2,9296,5
4,2,2318,3
...,...,...,...
5976474,49925,510,5
5976475,49925,528,4
5976476,49925,722,4
5976477,49925,949,5


In [ ]:
import scipy.sparse as sparse
from pandas.api.types import CategoricalDtype

user_index = ratings.user_id.unique()
books_index = ratings.book_id.unique()

rows = ratings['user_id'].astype(CategoricalDtype(categories=user_index)).cat.codes
cols = ratings['book_id'].astype(CategoricalDtype(categories=books_index)).cat.codes

matrix = sparse.csr_matrix((ratings.rating, (rows, cols)), shape=(len(user_index), len(books_index)))

matrix = matrix.toarray()

In [ ]:
for ind, score in enumerate(matrix[50]):
    if score>=4:
        book_id = books_index[ind]
        print(score, books[books['book_id']==book_id].original_title.values[0])


4 La sombra del viento
5 The Sun Also Rises
4 The Catcher in the Rye
4 Slaughterhouse-Five, or The Children's Crusade: A Duty-Dance with Death 
4 Life of Pi
4 Animal Farm: A Fairy Story
4 Nineteen Eighty-Four
4 Never Let Me Go
4 The Sea
4 Of Mice and Men 
4 Das Parfum. Die Geschichte eines Mörders
4 The Secret History
4 Motherless Brooklyn
4 In Cold Blood
4 Kitchen Confidential: Adventures in the Culinary Underbelly
4 Franny and Zooey
4 The Poisonwood Bible
4 Frankenstein; or, The Modern Prometheus
4 Howl and Other Poems 
4 The Hours
4 Under the Banner of Heaven: A Story of Violent Faith
5 The Omnivore's Dilemma
4 Fahrenheit 451
4 The Crucible
4 The Grapes of Wrath
4 Disgrace
4 The Little Friend
4 On the Road
4 Nesnesitelná lehkost bytí
4 The Bean Trees
4 L'Amant
5 Stiff: The Curious Lives of Human Cadavers
4 Candide
4 Angela's Ashes: A Memoir
4 The Keep
4 Lolita
4 Still Life with Woodpecker
4 Bastard Out of Carolina
5 Le Petit Prince
4 Se una notte d'inverno un viaggiatore
4 Siddharth

In [ ]:
viwed_arr = np.where(matrix[50]>0)[0]

In [ ]:
from sklearn.neighbors import NearestNeighbors

nbrs = NearestNeighbors(n_neighbors=15).fit(matrix)
search_vec = matrix[50]
closest_vec = matrix[nbrs.kneighbors(np.atleast_2d(search_vec))[1]]

In [ ]:
recommendations = []
for vec in closest_vec[0]:
    for ind, value in enumerate(vec):
        if value>=3:
            if ind not in viwed_arr:
                recommendations.append(ind)


In [ ]:
for ind in recommendations:
    book_id = books_index[ind]
    print(books[books['book_id']==book_id].original_title.values[0])

Shatter Me (Shatter Me, #1)
The Jordan Rules
The Hard Thing about Hard Things: Building A Business When There Are No Easy Answers
Freakonomics: A Rogue Economist Explores the Hidden Side of Everything
Rework
Quiet: The Power of Introverts in a World that Can't Stop Talking
From a Buick 8
The Long Walk
The Pillars of the Earth
Band of Brothers: E Company, 506th Regiment, 101st Airborne from Normandy to Hitler's Eagle's Nest
Stalingrad: The Fateful Siege, 1942–1943
The Making of the Atomic Bomb
Burned
Iced
Feverborn
Vicious
Harry Potter and the Prisoner of Azkaban
Harry Potter and the Half-Blood Prince
Harry Potter and the Order of the Phoenix
Harry Potter and the Chamber of Secrets
Harry Potter and the Goblet of Fire
Der Prozeß
Brave New World
Ten Little Niggers
Die Verwandlung
The Picture of Dorian Gray
Cathedral
رجال في الشمس
1/4 جرام
色彩を持たない多崎つくると、彼の巡礼の年 [Shikisai o motanai Tazaki Tsukuru to, kare no junrei no toshi]
History of Art
The Story of Art
The Art Forger


In [ ]:
#make mask for some users
user_indexes_for_masking = np.random.randint(matrix.shape[0], size=300)

In [ ]:
masked_result = {}
for usr_index in user_indexes_for_masking:
    for ind, value in enumerate(matrix[usr_index]):
        if value>=4:
            masked_result[(usr_index, ind)] = value
            matrix[usr_index][ind] = 0
            break

In [ ]:
from sklearn.neighbors import NearestNeighbors
nbrs = NearestNeighbors(n_neighbors=15).fit(matrix)


In [ ]:
matrix[50]

array([4, 0, 0, ..., 0, 0, 0])

In [ ]:
repair_grade = 0

for position in masked_result.keys():
    print(position)
    search_vec = matrix[position[0]]
    closest_vec = matrix[nbrs.kneighbors(np.atleast_2d(search_vec))[1]]
    for vec in closest_vec[0]:
        for ind, value in enumerate(vec):
            if value>=4:
                if ind not in viwed_arr:
                    recommendations.append(ind)
    for recs in recommendations:
        if recs == position[1]:
            repair_grade+=1
            break

(47955, 264)
(49481, 15)
(53209, 2619)
(10423, 28)
(48303, 7)
(18727, 7)
(52664, 15)
(35932, 78)
(21087, 5)
(40905, 12)
(29982, 37)
(19621, 28)
(34779, 5)
(16964, 7)
(51306, 15)
(33950, 81)
(16105, 304)
(31298, 3276)
(39559, 18)
(14583, 2)
(33103, 12)
(40569, 38)
(4538, 15)
(12082, 8)
(30388, 83)
(31439, 5)
(51876, 13)
(36105, 12)
(48441, 5)
(43125, 65)
(38079, 92)
(20223, 15)
(42673, 5)
(1804, 13)
(46264, 762)
(28946, 5)
(35558, 27)
(7451, 7)
(14815, 51)
(12775, 31)
(19792, 28)
(41103, 32)
(11314, 5)
(51782, 15)
(5756, 8)
(10929, 111)
(3581, 17)
(20568, 65)
(5839, 38)
(41066, 15)
(13575, 37)
(42774, 82)
(50604, 32)
(18955, 0)
(1049, 5)
(9592, 25)
(46207, 34)
(3987, 0)
(50281, 497)
(44214, 65)
(4564, 5)
(31371, 15)
(42601, 15)
(26522, 0)
(13774, 12)
(6779, 89)
(27963, 688)
(34780, 15)
(8277, 8)
(418, 5)
(4142, 31)
(30818, 0)
(8741, 12)
(775, 5)
(25642, 7)
(6902, 85)
(11172, 5)
(5790, 22)
(8708, 13)
(31779, 97)
(1, 1)
(1531, 5)
(27481, 7)
(32291, 0)
(2498, 6)
(41651, 15)
(48820, 5)
(727

In [ ]:
repair_grade/len(masked_result)

0.83